# Run MPI Hello World Examples

This tutorial introduces examples for running a simple MPI "Hello World" Fortran code using SLURM, both as an interactive job and as a batch job. 

```{note}
The files mentioned in this tutorial are accessible on the P-Cluster at `/efs_ecco/ECCO/misc/hello/`. Some of them are also provided as downloadable links within this tutorial.
``` 

## MPI Hello World Fortran Code

We use an MPI Fortran code called [hello.f](./hello/hello.f) in which each process prints out "Hello, world!".

## Create a Local Test Directory and Obtain the Relevant Files
Users first need to copy the directory `/efs_ecco/ECCO/misc/hello/` to a preferred location (e.g., `/home/USERNAME/mpitest/` in the example below, with `USERNAME` replaced by the actual username), and then change into `~/mpitest/hello/`, as shown in the following example:

```
echo "navigate to your home directory"
cd ~

echo "show your home directory "
pwd
/home/USERNAME

echo "make a directory for the mpi hello test files"
mkdir mpitest

echo "navigate into that directory"
cd mpitest

echo "copy the hello world mpi files here"
cp -pr /efs_ecco/ECCO/misc/hello .

echo "navigate  into that directory"
cd hello

echo "show contents
ls -1
```
Users should see the following:
```
README
hello
hello.f
hello_job_1140.log
hello_salloc.log
hello_sbatch.csh
```

## Compile
The executable, called `hello`, has been pre-gerenated. If needed, use the following command to re-compile the Fortran program `hello.f` and generate the executable:
```
mpiifort -O2 -o hello hello.f
```


<div class="alert alert-block alert-info"><b>Tip:</b> The "O" in -O2 is  letter "O", not Zero. `-O2` stands for Optimization level 2.</div>


## Run as an Interactive Job

To run the executable as an interactive job using 6 CPUs, use the following command to request the computing resoure:
```
salloc --ntasks=6 --ntasks-per-node=2 --partition=sealevel-c5xl-demand --time=00:05:00
```

Here, `salloc` requests 6 tasks with 2 tasks per node, effectively requesting 3 nodes from the partition `sealevel-c5xl-demand`. We only request 5 minute of wall clock time (`--time=00:05:00`) since the job runs quickly (the actual run completes in at most a couple of seconds). Although the job itself finishes quickly, it often takes a few minutes for the computing resources to become ready. During this period, users will see a message similar to the following:
```
salloc: Granted job allocation 1263
salloc: Waiting for resource configuration
```
When the allocation is granted, users will see the following message, and the prompt will appear:
```
salloc: Nodes sealevel-c5xl-demand-dy-c5xlarge-[1-3] are ready for job
```
Users can then use the following command to use of the node as the inactive node (replace the job ID `1263` with the actual job ID):
```
srun --jobid=1263 --pty /bin/bash
```
Uses will notice the prompt changes 
```
USERNAME@sealevel-c5xl-demand-dy-c5xlarge-1:/SOMEWORKINGDIR/$
```
Then, issue the following command to launch the executable `./hello` using 6 procrosses. 
```
mpirun -np 6 ./hello
```

The output of the interactive job is as follows:
```
salloc: Granted job allocation 1139
salloc: Waiting for resource configuration
salloc: Nodes sealevel-c5xl-demand-dy-c5xlarge-[1-3] are ready for job
  Process   1 says "Hello, world!"
12 May       2025  10:08:00.038 PM      

HELLO_MPI - Master process:
  FORTRAN77/MPI version
  A simple MPI program.

  The number of processes is   6

  Process   0 says "Hello, world!"

Elapsed wall clock time =   0.109248E-04 seconds.
  Process   2 says "Hello, world!"
  Process   3 says "Hello, world!"
  Process   4 says "Hello, world!"
  Process   5 says "Hello, world!"

HELLO_MPI:
  Normal end of execution.

12 May       2025  10:08:00.062 PM      
salloc: Relinquishing job allocation 1139
```

There are 6 printouts of "Hello, world!", corresponding to the 6 processors.

Issue the `exit` command times (see explanations below) and release the allocated resources:
```
exit
exit
```
The first `exit` exits the interactive shell started by the `srun` command, while the second `exit` ends the `salloc` session and releases the allocated resources. After the first "exit", you will see the prompt changes back to `USERNAME@ip-10-20-22-69:~$`. After the second `exit`, you will see the following message: 
```
scsalloc: Relinquishing job allocation 1139
```

If you now issue the command `squeue`, you will see your job ID is not longer in the job list.

<div class="alert alert-block alert-info"><b>Note:</b> If you look carefully, you'll see that the lines with 'Process  n says "Hello, world!" can be out of order (really scrambled) because how *nix machines flush the caches of text sent to STDOUT. It's not a big deal if the order of the 'Process  n says "Hello World"' is scrambled. If you see Hello Worlds!, you're good.</div>

## Run as a Batch Job

We can also run the executable as a batch job. To do this, we create a shell script called [hello_sbatch.csh](./hello/hello_sbatch.csh) to run the executable using 12 processors as a batch job. To submit it, issue the following command:
```
sbatch hello_sbatch.csh
```

The script first specifies some SLURM directives, such as `#SBATCH -J hello_sbatch`, which define the job submission parameters and are interpreted by SLURM when you submit the script using `sbatch`. It then sets the number of processors (`nprocs`) and use `mpirun` to lauch the executable. 

The output will be written to a file called `hello_job_NNNN.log` (where `NNNN` is the actual job number) in the directory where the job is submitted. You should see 12 printouts of "Hello, world!" in the `hello_job_NNNN.log` file. An example output file from the batch job is as follows:
```
  Process   1 says "Hello, world!"
  Process   2 says "Hello, world!"
  Process   3 says "Hello, world!"
  Process   4 says "Hello, world!"
  Process   5 says "Hello, world!"
13 May       2025   0:09:28.738 AM

HELLO_MPI - Master process:
  FORTRAN77/MPI version
  A simple MPI program.

  The number of processes is  12

  Process   0 says "Hello, world!"

Elapsed wall clock time =   0.138135E-05 seconds.
  Process   6 says "Hello, world!"
  Process   7 says "Hello, world!"
  Process   8 says "Hello, world!"
  Process   9 says "Hello, world!"
  Process  10 says "Hello, world!"
  Process  11 says "Hello, world!"

HELLO_MPI:
  Normal end of execution.

13 May       2025   0:09:28.824 AM
```

See, here the Hello, world! statements are also jumbled up.  No big deal, delcare success and move on.